In [36]:
import os

os.environ["NEO4J_URI"] = "neo4j+s://e4aa044e.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "NmObplaapQCdBkswnB2aQJPmlunmvH7LU4xabbP489Y"

In [37]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

In [38]:
from langchain.document_loaders import WikipediaLoader

# Read the wikipedia article
raw_documents = WikipediaLoader(query="Elizabeth I").load()

In [39]:
from langchain.text_splitter import TokenTextSplitter

# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size = 512, chunk_overlap = 24)
documents = text_splitter.split_documents(raw_documents[:3])

In [40]:
from langchain_ollama.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm = ChatOllama(model = "llama3.1:8b-instruct-fp16")
llm_transformer = LLMGraphTransformer(llm = llm)

# Extract graph data
graph_documents = llm_transformer.convert_to_graph_documents(documents)

# Store to neo4j
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel = True,
    include_source = True
)

In [41]:
from langchain_community.vectorstores import Neo4jVector
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model = "llama3.1:8b-instruct-fp16")

vector_index = Neo4jVector.from_existing_graph( # Same as vector_store
    embeddings,
    search_type = "hybrid",
    node_label = "Document",
    text_node_properties = ["text"],
    embedding_node_property = "embedding"
)

In [42]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

# Extract entities from the text using Pydantic Schema
class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
    ...,
    description="All the person, organization, or business entities that appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following"
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

In [43]:
# Testing it out
entity_chain.invoke({"question": "Where was Amelia Earhart born?"})

Entities(names=['Amelia Earhart'])

In [53]:
from langchain_neo4j.vectorstores.neo4j_vector import remove_lucene_chars # New package, old remove_lucene_chars is deprecated

graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text
    search. It processes the input string by splitting it into words and 
    appending a similarity threshold (~2 changed characters) to each
    word, then combines them using the AND operator. Useful for mapping
    entities from user questions to database values, and allows for some 
    misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

In [73]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})      # Adding WITH node makes the query more accurate, remove ! from MENTIONS
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, 
            {limit:2})
            YIELD node,score
            CALL {
              
              MATCH (node)-[r:MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS 
              output
              UNION
              
              MATCH (node)<-[r:MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS 
              output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [74]:
print(structured_retriever("Who is Elizabeth I?"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 4, column: 13, offset: 117} for query: "CALL db.index.fulltext.queryNodes('entity', $query, \n            {limit:2})\n            YIELD node,score\n            CALL {\n              \n              MATCH (node)-[r:MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS \n              output\n              UNION\n              \n              MATCH (node)<-[r:MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS \n              output\n            }\n            RETURN output LIMIT 50\n            "


2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Elizabeth I
2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Henry Viii
2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Anne Boleyn
2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Edward Vi
2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Lady Jane Grey
2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Mary I
2912ff1bf15ba29f11702607f35da890 - MENTIONS -> William Cecil
2912ff1bf15ba29f11702607f35da890 - MENTIONS -> James Vi Of Scotland
2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Mary, Queen Of Scots
2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Sir Francis Walsingham
2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Mary
442d5116ab76d2c40f934a2bf47b88e0 - MENTIONS -> Henry Viii
442d5116ab76d2c40f934a2bf47b88e0 - MENTIONS -> Elizabeth
442d5116ab76d2c40f934a2bf47b88e0 - MENTIONS -> Greenwich Palace
442d5116ab76d2c40f934a2bf47b88e0 - MENTIONS -> Henry Viii Of England
442d5116ab76d2c40f934a2bf47b88e0 - MENTIONS -> An Boleyn
442d5116ab76d2c40f934a2bf47

In [75]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [92]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import Tuple, List, Optional

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

# Defining the search query
_search_query = RunnableBranch(
    # If input includes chat_history, condense
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOllama(model = "llama3.1:8b-instruct-fp16")
        | StrOutputParser(),
    ),
    # Else pass the question
    RunnableLambda(lambda x : x["question"]),
)

In [88]:


template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [89]:
chain.invoke({"question": "Which house did Elizabeth I belong to?"})

Search query: Which house did Elizabeth I belong to?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 4, column: 13, offset: 117} for query: "CALL db.index.fulltext.queryNodes('entity', $query, \n            {limit:2})\n            YIELD node,score\n            CALL {\n              \n              MATCH (node)-[r:MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS \n              output\n              UNION\n              \n              MATCH (node)<-[r:MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS \n              output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Ne

'The Tudor dynasty.'

In [90]:
chain.invoke(
    {
        "question": "When was she born?",
        "chat_history": [("Which house did Elizabeth I belong to?",
        "House Of Tudor")],
    }
)

NameError: name '_format_chat_history' is not defined